## Import and Read in Data

In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
FILE_NAME = "preprocess.ipynb"
PARENT_DIR = os.path.abspath(os.path.join(os.path.dirname(FILE_NAME), "."))
app_train = pd.read_csv( PARENT_DIR + '/data/application_train.csv')
app_test = pd.read_csv( PARENT_DIR + '/data/application_test.csv')

## Encode
### Column Type
各特征的类型

In [ ]:
# Number of each type of column
app_train.dtypes.value_counts()

In [ ]:
# 每个标称特征有几个标称
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

### Encode the Categorical/Object Features
01编码，one-hot编码

In [ ]:
## TODO 

## Missing Values
### Calculate miss-rate of all features
计算两个数据集各特征的缺失值比例

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values_train = missing_values_table(app_train)
missing_values_train.head(20)
missing_values_test = missing_values_table(app_test)
missing_values_test.head(20)

## Anomalies(异常值)
### In `DAYS_BIRTH` column

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()
# 结果无问题

### In `DAYS_EMPLOYED` column

In [ ]:
app_train['DAYS_EMPLOYED'].describe()
# 有一个有问题的正数

In [ ]:
# statics about anomalies of `days_employed`
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

#### Turn the anomalies into NAN

In [ ]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = (app_train["DAYS_EMPLOYED"] == 365243)

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram')
plt.xlabel('Days Employment')

#### Do the same thing in Test Dataset

In [ ]:
# Add a column 'DAYS_EMPLOYED_ANOM' to tell if 'DAYS_EMPLOYED' is anomaly.
app_test['DAYS_EMPLOYED_ANOM'] = (app_test["DAYS_EMPLOYED"] == 365243)
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))